In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import oracledb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from matplotlib.ticker import StrMethodFormatter,FormatStrFormatter
pd.options.display.float_format = '{:.5f}'.format
plt.style.use("seaborn-whitegrid")
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer
import datetime as dt
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

# data load

In [39]:
school_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/input_df_초중등.csv",encoding='utf-8')
bus_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/input_df_bus.csv",encoding='utf-8')
con_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/input_df_shop.csv",encoding='utf-8')
sub_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/지하철.csv",encoding='utf-8')
newdata=pd.read_csv("/content/drive/MyDrive/A하금티/jeonse_data_processed.csv",encoding='utf-8')

crime_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/crime.csv",encoding='utf-8')
police_df=pd.read_csv("/content/drive/MyDrive/A하금티/data/police.csv",encoding='utf-8')

In [31]:
newdata.isnull().sum()

Unnamed: 0       0
cggCd            0
stdgCd           0
mno              0
sno              0
flr              0
ctrtDay          0
rentArea         0
grfe             0
rtfe             0
bldgUsg          0
newUpdtYn        0
ctrtUpdtUseYn    0
bfrGrfe          0
lat              0
lon              0
자치구명_강남구         0
자치구명_강서구         0
자치구명_관악구         0
자치구명_동작구         0
자치구명_송파구         0
ctrtDayMonth     0
dtype: int64

In [45]:
# newdata.drop(columns=['Unnamed: 0'],inplace=True)

## 데이터 전처리

In [46]:
input_df=newdata

In [57]:
  # 신규계약구분을 1과 0으로 변환하는 함수
def convert_new_contract_type(contract_type):
    if contract_type == '신규':
        return 1
    elif contract_type == '갱신':
        return 0
    else:
        return None  # 예외 처리
# 날짜 형식을 통일하는 함수
def format_date(date_str):
    if len(date_str.split('.')[0]) == 2:
        return '20' + date_str
    return date_str

In [49]:
# bldgUsg 변환함수
def convert_new_bldgUsg(bldgUsg):
    if bldgUsg == '아파트':
        return 1
    elif bldgUsg == '연립다세대':
        return 2
    else:
        return 3  # 예외 처리

input_df['bldgUsg'] = input_df['bldgUsg'].apply(convert_new_bldgUsg)

In [54]:
def transfer(input_df):
  input_df.drop(columns='bfrRtfe',inplace=True)
  # 종전임대료 drop
  input_df.drop(columns='stdgNm',inplace=True) #법정동명 drop

  #자치구명 원핫 인코더
  # 자치구명 리스트
  districts = ['강남구', '강서구', '관악구', '동작구', '송파구']

  # 원핫 인코딩 컬럼 생성
  for district in districts:
      input_df[f'자치구명_{district}'] = (input_df['cggNm'] == district).astype(int)

  # input_df.drop(columns='rtfe',inplace=True)
  #임대료 drop
  input_df.drop(columns='lotnoSe',inplace=True) #지번구분 drop
  input_df.drop(columns='archYr',inplace=True) #건축년도 drop
  input_df.drop(columns='lotnoSeNm',inplace=True) #전월세 구분 drop 다 전세여서
  input_df.drop(columns='bldgNm',inplace=True)  # 건물명 drop
  # input_df.drop(columns='ctrtPrd',inplace=True)
  # ctrtPrd
  input_df.drop(columns='rcptYr',inplace=True)  # 계약기간 drop
  input_df.drop(columns='rentSe',inplace=True)  # rentSe drop

  # apply 함수를 사용하여 변환 적용
  input_df['newUpdtYn'] = input_df['newUpdtYn'].apply(convert_new_contract_type)

  #계약일 날짜로 변환
  current =  datetime.now()
  input_df["ctrtDay"] = current - pd.to_datetime(input_df["ctrtDay"],format='%Y%m%d')
  input_df["ctrtDay"] = input_df["ctrtDay"].dt.days

  input_df['시작일'] = input_df['ctrtPrd'].str.split('~').str[0]
  input_df['종료일'] = input_df['ctrtPrd'].str.split('~').str[1]

  input_df = input_df.dropna(subset=['시작일'])
  input_df = input_df.dropna(subset=['종료일'])

  # "."으로 시작하는 데이터 제거
  input_df = input_df[~input_df['시작일'].str.startswith('.')]
  input_df = input_df[~input_df['종료일'].str.startswith('.')]

  # 날짜 형식을 통일
  input_df['시작일'] = input_df['시작일'].apply(format_date)
  input_df['종료일'] = input_df['종료일'].apply(format_date)

  # 문자열을 datetime 형식으로 변환
  input_df['시작일'] = pd.to_datetime(input_df['시작일'], format="%Y.%m")
  input_df['종료일'] = pd.to_datetime(input_df['종료일'], format="%Y.%m")

  # 두 날짜 사이의 차이 계산 (월 단위)
  input_df['ctrtDayMonth'] = (input_df['종료일'].dt.year - input_df['시작일'].dt.year) * 12 + (input_df['종료일'].dt.month - input_df['시작일'].dt.month)
  input_df.drop(columns='시작일',inplace=True)
  input_df.drop(columns='종료일',inplace=True)
  input_df.drop(columns='ctrtPrd',inplace=True)
  input_df['ctrtDayMonth']

  input_df['ctrtUpdtUseYn'] = input_df['ctrtUpdtUseYn'].apply(lambda x: 1 if x == '○' else 0)
  input_df = input_df.dropna(subset=['newUpdtYn'])
  input_df = input_df.dropna(subset=['bfrGrfe'])
  return input_df

In [ ]:
school_df=transfer(school_df)
newdata=transfer(newdata)

정제된 본 데이터

### 추가 데이터 매핑

In [82]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56471 entries, 0 to 56470
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                56471 non-null  int64  
 1   cggCd                     56471 non-null  int64  
 2   stdgCd                    56471 non-null  int64  
 3   mno                       56471 non-null  int64  
 4   sno                       56471 non-null  int64  
 5   flr                       56471 non-null  float64
 6   ctrtDay                   56471 non-null  int64  
 7   rentArea                  56471 non-null  float64
 8   grfe                      56471 non-null  int64  
 9   rtfe                      56471 non-null  int64  
 10  bldgUsg                   56471 non-null  int64  
 11  newUpdtYn                 56471 non-null  float64
 12  ctrtUpdtUseYn             56471 non-null  int64  
 13  bfrGrfe                   56471 non-null  float64
 14  lat   

In [124]:
newdata = newdata.reset_index(drop=True)
school_df = school_df.reset_index(drop=True)
bus_df = bus_df.reset_index(drop=True)
con_df = con_df.reset_index(drop=True)
sub_df = sub_df.reset_index(drop=True)

# 데이터프레임 결합
df1 = pd.concat([newdata, school_df['500m_내_초중등수']], axis=1)
df2 = pd.concat([df1, bus_df['500m_내_bus수']], axis=1)
df3 = pd.concat([df2, con_df['500m_내_마트수']], axis=1)
df4 = pd.concat([df3, sub_df['Closest_Station_Distance']], axis=1)

In [ ]:
# 필터링할 자치구명
desired_gu = ['관악구', '송파구', '강서구', '강남구', '동작구']

# 자치구명 필터링
crime_df = crime_df[newdata['cggNm'].isin(desired_gu)]

# 결과 출력
print(crime_df)

In [173]:
newdata=pd.read_csv("/content/drive/MyDrive/A하금티/jeonse_data_processed.csv",encoding='utf-8')

In [174]:
crime=newdata.merge(crime_df, how='left', left_on='cggNm', right_on='자치구별')
newdata=transfer(newdata)
crime=transfer(crime)
crime=crime[['발생','검거']]
crime

,발생,검거
0,4879,3411
1,4879,3411
2,4879,3411
3,4879,3411
4,4879,3411
...,...,...
69423,4663,3591
69424,4663,3591
69425,4663,3591
69426,4663,3591


In [178]:
crime = crime.reset_index(drop=True)
df4 = df4.reset_index(drop=True)

In [179]:
df5=pd.concat([crime,df4],axis=1)
df5

,발생,검거,cggCd,stdgCd,mno,sno,flr,ctrtDay,rentArea,grfe,...,자치구명_강남구,자치구명_강서구,자치구명_관악구,자치구명_동작구,자치구명_송파구,ctrtDayMonth,500m_내_초중등수,500m_내_bus수,500m_내_마트수,Closest_Station_Distance
0,4879,3411,11620,10300,1061,18,14.00000,370,28.05000,16500,...,0,0,1,0,0,12,0,10,1,0.22666
1,4879,3411,11620,10200,1435,7,2.00000,553,19.50000,10000,...,0,0,1,0,0,24,1,28,2,0.26297
2,4879,3411,11620,10200,1462,12,4.00000,506,21.84000,13000,...,0,0,1,0,0,24,1,27,0,0.51707
3,4879,3411,11620,10100,1564,1,3.00000,509,17.40000,7000,...,0,0,1,0,0,24,3,20,1,0.37567
4,4879,3411,11620,10200,1419,9,3.00000,468,27.32000,15000,...,0,0,1,0,0,12,1,32,2,0.29274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56466,4663,3591,11500,10300,424,23,4.00000,218,31.68000,16000,...,0,1,0,0,0,24,0,16,1,0.43580
56467,4663,3591,11500,10300,105,357,5.00000,218,24.30000,21500,...,0,1,0,0,0,24,3,26,2,0.17364
56468,4663,3591,11500,10300,1091,0,11.00000,218,84.94000,61000,...,0,1,0,0,0,24,6,4,0,0.63036
56469,4663,3591,11500,10300,410,296,4.00000,218,46.77000,12500,...,0,1,0,0,0,24,1,35,0,0.69853


In [182]:
df5.cggCd #넣는게 나을지 모르겠음

0        11620
1        11620
2        11620
3        11620
4        11620
         ...  
56466    11500
56467    11500
56468    11500
56469    11500
56470    11590
Name: cggCd, Length: 56471, dtype: int64

In [217]:
newdata=pd.read_csv("/content/drive/MyDrive/A하금티/jeonse_data_processed.csv",encoding='utf-8')
newdata

,rcptYr,cggCd,cggNm,stdgCd,stdgNm,lotnoSe,lotnoSeNm,mno,sno,flr,...,bldgNm,archYr,bldgUsg,ctrtPrd,newUpdtYn,ctrtUpdtUseYn,bfrGrfe,bfrRtfe,lat,lon
0,2023,11620,관악구,10300,남현동,1,대지,1061,18,14.00000,...,(1061-18),2005.00000,오피스텔,23.08~24.08,신규,NaN,0.00000,NaN,37.47450,126.98139
1,2023,11620,관악구,10200,신림동,1,대지,1435,7,2.00000,...,(1435-7),2003.00000,오피스텔,23.02~25.02,신규,NaN,0.00000,NaN,37.48561,126.92733
2,2023,11620,관악구,10200,신림동,1,대지,1462,12,4.00000,...,(1462-12),2003.00000,오피스텔,23.02~25.02,갱신,NaN,13000.00000,NaN,37.48624,126.92446
3,2023,11620,관악구,10100,봉천동,1,대지,1564,1,3.00000,...,(1564-1),2004.00000,오피스텔,23.02~25.02,갱신,NaN,7000.00000,NaN,37.47977,126.94463
4,2023,11620,관악구,10200,신림동,1,대지,1419,9,3.00000,...,(1419-9),2012.00000,오피스텔,23.04~24.04,갱신,NaN,11000.00000,NaN,37.48661,126.93106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69424,2023,11500,강서구,10300,화곡동,1,대지,105,357,5.00000,...,화곡갤럭시오피스텔,2018.00000,연립다세대,23.12~25.12,신규,NaN,0.00000,NaN,37.54202,126.84232
69425,2023,11500,강서구,10300,화곡동,1,대지,1091,0,11.00000,...,화곡푸르지오,2002.00000,아파트,23.12~25.12,갱신,NaN,65000.00000,NaN,37.54462,126.83144
69426,2023,11500,강서구,10300,화곡동,1,대지,410,296,4.00000,...,화성빌라41차,2001.00000,연립다세대,24.01~26.01,신규,NaN,0.00000,NaN,37.53760,126.84966
69427,2023,11590,동작구,10500,흑석동,1,대지,335,0,10.00000,...,흑석한강센트레빌,2011.00000,아파트,24.02~26.02,신규,NaN,0.00000,NaN,37.50575,126.96534


In [193]:
newdata.ctrtDay.min()

20221106

In [197]:
inte_rate=pd.read_csv("/content/drive/MyDrive/A하금티/data/inte_rate.csv",encoding='cp949')
inte_rate=inte_rate[['TIME','value']]
inte_rate

,TIME,value
0,20221106,3.00000
1,20221107,3.00000
2,20221108,3.00000
3,20221109,3.00000
4,20221110,3.00000
...,...,...
630,20240704,3.50000
631,20240705,3.50000
632,20240706,3.50000
633,20240707,3.50000


In [201]:
newdata.ctrtDay

0        20230708
1        20230106
2        20230222
3        20230219
4        20230401
           ...   
69424    20231207
69425    20231207
69426    20231207
69427    20231207
69428    20231207
Name: ctrtDay, Length: 69429, dtype: int64

In [210]:
inte_rate=inte_rate.drop_duplicates(['TIME'], keep='first')

In [218]:
newdata=newdata.merge(inte_rate, how='left', left_on='ctrtDay', right_on='TIME')
newdata=transfer(newdata)
newdata

,cggCd,stdgCd,mno,sno,flr,ctrtDay,rentArea,grfe,rtfe,bldgUsg,...,lat,lon,TIME,value,자치구명_강남구,자치구명_강서구,자치구명_관악구,자치구명_동작구,자치구명_송파구,ctrtDayMonth
0,11620,10300,1061,18,14.00000,370,28.05000,16500,0,오피스텔,...,37.47450,126.98139,20230708,3.50000,0,0,1,0,0,12
1,11620,10200,1435,7,2.00000,553,19.50000,10000,0,오피스텔,...,37.48561,126.92733,20230106,3.25000,0,0,1,0,0,24
2,11620,10200,1462,12,4.00000,506,21.84000,13000,0,오피스텔,...,37.48624,126.92446,20230222,3.50000,0,0,1,0,0,24
3,11620,10100,1564,1,3.00000,509,17.40000,7000,4,오피스텔,...,37.47977,126.94463,20230219,3.50000,0,0,1,0,0,24
4,11620,10200,1419,9,3.00000,468,27.32000,15000,0,오피스텔,...,37.48661,126.93106,20230401,3.50000,0,0,1,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69423,11500,10300,424,23,4.00000,218,31.68000,16000,0,연립다세대,...,37.53139,126.84178,20231207,3.50000,0,1,0,0,0,24
69424,11500,10300,105,357,5.00000,218,24.30000,21500,0,연립다세대,...,37.54202,126.84232,20231207,3.50000,0,1,0,0,0,24
69425,11500,10300,1091,0,11.00000,218,84.94000,61000,0,아파트,...,37.54462,126.83144,20231207,3.50000,0,1,0,0,0,24
69426,11500,10300,410,296,4.00000,218,46.77000,12500,0,연립다세대,...,37.53760,126.84966,20231207,3.50000,0,1,0,0,0,24


In [219]:
newdata = newdata.reset_index(drop=True)
newdata=newdata[['value']]
df6=pd.concat([newdata,df5],axis=1)
df6

,value,발생,검거,cggCd,stdgCd,mno,sno,flr,ctrtDay,rentArea,...,자치구명_강남구,자치구명_강서구,자치구명_관악구,자치구명_동작구,자치구명_송파구,ctrtDayMonth,500m_내_초중등수,500m_내_bus수,500m_내_마트수,Closest_Station_Distance
0,3.50000,4879,3411,11620,10300,1061,18,14.00000,370,28.05000,...,0,0,1,0,0,12,0,10,1,0.22666
1,3.25000,4879,3411,11620,10200,1435,7,2.00000,553,19.50000,...,0,0,1,0,0,24,1,28,2,0.26297
2,3.50000,4879,3411,11620,10200,1462,12,4.00000,506,21.84000,...,0,0,1,0,0,24,1,27,0,0.51707
3,3.50000,4879,3411,11620,10100,1564,1,3.00000,509,17.40000,...,0,0,1,0,0,24,3,20,1,0.37567
4,3.50000,4879,3411,11620,10200,1419,9,3.00000,468,27.32000,...,0,0,1,0,0,12,1,32,2,0.29274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56466,3.50000,4663,3591,11500,10300,424,23,4.00000,218,31.68000,...,0,1,0,0,0,24,0,16,1,0.43580
56467,3.50000,4663,3591,11500,10300,105,357,5.00000,218,24.30000,...,0,1,0,0,0,24,3,26,2,0.17364
56468,3.50000,4663,3591,11500,10300,1091,0,11.00000,218,84.94000,...,0,1,0,0,0,24,6,4,0,0.63036
56469,3.50000,4663,3591,11500,10300,410,296,4.00000,218,46.77000,...,0,1,0,0,0,24,1,35,0,0.69853


In [220]:
df6.to_csv("/content/drive/MyDrive/A하금티/data/finaldf.csv")